In [ ]:
import os 
import json
import pandas as pd

from src.download.patient import PatientDownload
from src.download.luftdata_downloader import AirdataDownloader
from src.geolocation.address_transformation import get_long_lat_from_postal_code
from src.utils.graph_configurations import get_index_timeline_plot

In [ ]:
# Load Patient Data - test data
file = open("data/raw/mii_testdaten.json")
data = json.load(file)

In [ ]:
# Extract Patient Data
patients = []

for entry in data["entry"]: 
    patient = PatientDownload().extract_patient(entry = entry)
    if patient:
        patients.append(patient)

In [ ]:
# add luftdaten API data + visualize index timeline data

air_data = AirdataDownloader()

for patient in patients:
    closest_station = air_data.get_closest_station(lat = patient.address.latitude, long = patient.address.longitude)

    # index
    daily_data_index_mean = air_data.get_luftdaten_index(station_id = closest_station["station"]["id"], start_date = "2019-01-01", end_date = "2019-12-31")
    get_index_timeline_plot(daily_data_index_mean, closest_station)
    
    daily_data_schadstoffe = air_data.get_luftdaten_schadstoffe(station_id = closest_station["station"]["id"], start_date = "2019-01-01", end_date = "2019-12-31")
    daily_data_schadstoffe = pd.DataFrame(daily_data_schadstoffe)

checks todo: 
- reihenfolge anpassen -> wenn in einem None dann wird wenn Wert bei anderen Datum hinten angehängt
- funktion sollte auch mean und durschnitt/ median in einem bestimmten Zeitraum zurückgeben
- Filter nach Luftqualitätsindex

## Looking at an individual patient

In [ ]:
our_patient = patients[0]

In [ ]:
print(f"The selected patient with the id '{our_patient.id}' was born on {our_patient.birth_date} and has the gender {our_patient.gender}.")